<a href="https://colab.research.google.com/github/shriprasad15/RNN-Course-Recommendation-System/blob/main/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_term_details = pd.read_excel("/content/drive/MyDrive/Foundation_validation_data.xlsx",sheet_name=0,header=1)
df_course_details = pd.read_excel("/content/drive/MyDrive/Foundation_validation_data.xlsx",sheet_name=1)

In [ ]:
len(df_term_details['city'].unique())
df_course_details.head()
## Dropping Unwanted Columns
cols_drop=["encoded smail",
           "Current Status","CurrentLevelAsOfTerm",
           "CurrentLevel","Diploma completed",
           "Degree completed","Theory Completed",
           "Projects completed","No Of Term Register",
           "Term Status.1","NOC Reg.1",
           "Term Status.2","NOC Reg.2","Term Status.3",
           "NOC Reg.3","Term Status.4","NOC Reg.4",
           "Term Status.5","NOC Reg.5","Projects.1",
           "Term Status.6","NOC Reg.6","Projects.2",
           "Term Status.7","NOC Reg.7","Projects",
           "Term Status.8","Term Status",
           "NOC Reg","DPDiplomaCertCompletion",
           "DSDiplomaCertCompletion","LatestDiplomaCompletedTerm",
           "DiplomaStatus","StudentStatus","IncomeLevel",
           "pwd","DS CGPA","DP CGPA","Diploma CGPA",
           "Degree CGPA","BSc completion","BS Completion","Unnamed: 57"]
df_term_details = df_term_details.drop(cols_drop,axis=1)

## Dropping all rows where students have not completed foundation
df_term_details = df_term_details.dropna(subset=["FoundationalCertCompletion"])
df_term_details.fillna(0)
df_course_details.drop(['encoded smail'],axis=1,inplace=True)

## Mapping Each term with a numeric value for calculation foundation completion time
term_mapping = {'F1-2021': 1, 'F1-2022': 4, 'F2-2021': 2, 'F2-2022': 5, 'F3-2021': 3, 'F3-2022': 6, 'F1-2023': 7,'F2-2023': 8,'F3-2023': 9}
## Mapping each grade with its respective numberic quantity
grade_mapping = {'S':10,'A':9,'B':8,'C':7,'D':6,'E':4,'U':0 , 'I':0, 'WA':0, 'WQ':0, 'I_OP':0,'I_BOTH':0, 'F':0, 'P':0}
## Replace the Term with its numberic values for the Qualifier Cleared Term
df_term_details['qual_cleared_term'] = df_term_details['qual_cleared_term'].replace(term_mapping)


## Replace the Term with its numberic values for the Foundation completed Term
df_term_details['FoundationalCertCompletion']= df_term_details['FoundationalCertCompletion'].replace(term_mapping)

df_term_details['Foundation CGPA'] = df_term_details['Foundation CGPA'].astype(float)


# Joining the Table Student Term details with Student Course Details
# by merging on the encoded email key . By this we get the details of all the courses taken
# by a student in a term with the grade and marks scored in the course.

df = pd.merge(df_term_details,df_course_details,left_on='encoded email',right_on='encoded (email)',how='left')

## Mapping Grade with its numeric quantity and selecting only those with courses with grade more that C
df['grade'] = df['grade'].replace(grade_mapping)
df = df[df['Foundation completed'] == 8]
df = df[df['Foundation CGPA']>7]

# Dropping the encoded email
df.drop(['encoded (email)'],axis=1,inplace=True)

# Replace Term with its numeric value
df['term_code']=df['term_code'].replace(term_mapping)

<ipython-input-3-30db5c0c753f>:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_term_details.fillna(0)
<ipython-input-3-30db5c0c753f>:33: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_term_details['qual_cleared_term'] = df_term_details['qual_cleared_term'].replace(term_mapping)
<ipython-input-3-30db5c0c753f>:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.s

In [ ]:
df2 = df[['encoded email','course_code','grade']]

In [ ]:
df.drop(['Country','State','city','Category','gender'],axis=1,inplace=True)


In [ ]:
df2 = df2[df2['grade']!=0]
df2.grade.unique()

array([ 8,  6, 10,  9,  7,  4])

In [ ]:
df2[df['encoded email']=='12jkfdsfl100009706']

<ipython-input-7-461283375ffd>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2[df['encoded email']=='12jkfdsfl100009706']


,encoded email,course_code,grade
81471,12jkfdsfl100009706,HS1001,10
81472,12jkfdsfl100009706,MA1004,7
81473,12jkfdsfl100009706,MA1002,8
81474,12jkfdsfl100009706,CS1002,10
81475,12jkfdsfl100009706,MA1003,4
81476,12jkfdsfl100009706,CS1001,10
81477,12jkfdsfl100009706,HS1002,10
81478,12jkfdsfl100009706,MA1001,10


In [ ]:
pivot_df = df.pivot_table(index='encoded email', columns='term_code', values=['course_code', 'grade'], aggfunc=lambda x: ' '.join(str(v) for v in x))

In [ ]:
!pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357291 sha256=f86f3c9fdff914ee6ccdee63bc78ef05014c1bb28cdb07a43cf085d7479547b2
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:

# Import necessary libraries
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split

# Load the built-in Movielens dataset
# data = Dataset.load_builtin('ml-100k')  # This is a smaller version for quick testing



In [ ]:
reader = Reader(rating_scale=(7, 10))

# Loads Pandas dataframe
data = Dataset.load_from_df(df2[["encoded email", "course_code", "grade"]], reader)


In [ ]:
df2.head()

,encoded email,course_code,grade
19,12jkfdsfl100019867,HS1002,8
20,12jkfdsfl100019867,MA1003,6
21,12jkfdsfl100019867,MA1002,6
22,12jkfdsfl100019867,CS1002,10
23,12jkfdsfl100019867,HS1001,8


In [ ]:
df2.shape

(48334, 3)

In [ ]:
# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)

# Define the similarity measure to use for the KNN algorithm
# We use 'cosine' similarity measure and user-based collaborative filtering
sim_options = {
    'name': 'cosine',
    'user_based': True  # User-based collaborative filtering
}

# Initialize the KNN algorithm
algo = KNNWithMeans(sim_options=sim_options)

# Train the algorithm on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)

# Evaluate the performance of the algorithm
accuracy.rmse(predictions)


# Function to get top N recommendations for a user
def get_top_n_recommendations(predictions, n=10):
    # First, map the predictions to each user
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the top n items
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

# Get the top 10 recommendations for each user
top_n_recommendations = get_top_n_recommendations(predictions, n=10)

# Print the top 10 recommendations for a specific user
user_id = '12jkfdsfl100009706'  # Change this to a valid user ID in your dataset
print(f"Top 10 recommendations for user {user_id}:")
for iid, est in top_n_recommendations[user_id]:
    print(f"Item {iid} with estimated rating {est:.2f}")



Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2455
Top 10 recommendations for user 12jkfdsfl100009706:
Item MA1003 with estimated rating 8.26


In [ ]:
accuracy

<module 'surprise.accuracy' from '/usr/local/lib/python3.10/dist-packages/surprise/accuracy.py'>